Installation is simple:

```
Pkg.add("Stan")
```

You need to install CmdStan first though. Get it from [here](http://mc-stan.org/users/interfaces/cmdstan). You also need to set the `CMDSTAN_HOME` environment variable, which should point to the folder in which you unpacked `CmdStan`.

This notebook assumes that you have created a `CmdStanDir` variable (mostly likely by setting it in your `~/.juliarc.jl` file) and that it points to the location in which you unpacked CmdStan.

In [1]:
using Stan

# Use the describe function from Mamba.
#
using Mamba

Define a model.

In [3]:
const bernoullistanmodel = "
data {
    int<lower=0> N;
    int<lower=0,upper=1> y[N];
} 
parameters {
    real<lower=0,upper=1> theta;
} 
model {
    theta ~ beta(1,1);
    y ~ bernoulli(theta);
}
";

Create a `Stanmodel` object. This specifies the basic attributes for the model.

In [4]:
stanmodel = Stanmodel(
    name="bernoulli",
    model=bernoullistanmodel,
    nchains=4
    );

Take a look at the model object.

In [5]:
stanmodel |> display

  name =                    "bernoulli"
  nchains =                 4
  num_samples =             1000
  num_warmup =                   1000
  thin =                    1
  useMamba =                true
  mambaThinning =           1
  monitors =                String[]
  model_file =              "bernoulli.stan"
  data_file =               ""
  output =                  Output()
    file =                    ""
    diagnostics_file =        ""
    refresh =                 100
  method =                  Sample()
    num_samples =             1000
    num_warmup =              1000
    save_warmup =             false
    thin =                    1
    algorithm =               HMC()
      engine =                  NUTS()
        max_depth =               10
      metric =                  Stan.diag_e
      stepsize =                1

.0
      stepsize_jitter =         1.0
    adapt =                   Adapt()
      gamma =                   0.05
      delta =                   0.8
      kappa =                   0.75
      t0 =                      10.0
      init_buffer =             75
      term_buffer =             50
      window =                  25


Interrogate attributes of the model.

In [6]:
stanmodel.method

  method =                  Sample()
    num_samples =             1000
    num_warmup =              1000
    save_warmup =             false
    thin =                    1
    algorithm =               HMC()
      engine =                  NUTS()
        max_depth =               10
      metric =                  Stan.diag_e
      stepsize =                1.0
      stepsize_jitter =         1.0
    adapt =                   Adapt()
      gamma =                   0.05
      delta =                   0.8
      kappa =                   0.75
      t0 =                      10.0
      init_buffer =             75
      term_buffer =             50
      window =                  25


Update characteristics of model. The `Stanmodel` object can be tailored either via the constructor or after the fact.

In [7]:
stanmodel.method.adapt.delta = 0.85

0.85

Set up the observed input data.

In [8]:
const bernoullidata = Dict("N" => 10, "y" => [0, 1, 0, 1, 0, 0, 0, 0, 0, 1])

Dict{String,Any} with 2 entries:
  "N" => 10
  "y" => [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]

Build the code and run the simulation. Uses `make` to recompile code, but only if necessary. Multiple chains are run in parallel (if possible).

In [9]:
rc, simulated = stan(stanmodel, [bernoullidata]);


make: '/home/colliera/proj/Z-317-talk-first-steps-stan/julia/tmp/bernoulli' is up to date.

Length of data array is not equal to nchains,
all chains will use the first data dictionary.

Calling /opt/cmdstan-2.17.1/bin/stansummary to infer across chains.

Inference for Stan model: bernoulli_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.012, 0.012, 0.010, 0.011) seconds, 0.044 seconds total
Sampling took (0.018, 0.025, 0.022, 0.024) seconds, 0.090 seconds total

                Mean     MCSE  StdDev    5%   50%   95%  N_Eff  N_Eff/s    R_hat
lp__            -8.2  2.1e-02    0.72  -9.6  -7.9  -7.6   1207    13479  1.0e+00
accept_stat__   0.94  1.5e-03   0.096  0.73  0.98   1.0   4000    44682  1.0e+00
stepsize__      0.87  5.0e-02   0.070  0.77  0.89  0.97    2.0       22  4.2e+13
treedepth__      1.4  9.0e-03    0.51   1.0   1.0   2.0   3189    35622  1.0e+00
n_leapfrog__     2.8  3.8e-02     1.6   1.0   3.

In [10]:
stanmodel.command

4-element Array{Base.AbstractCmd,1}:
 `./bernoulli sample num_samples=1000 num_warmup=1000 save_warmup=0 thin=1 adapt engaged=1 gamma=0.05 delta=0.85 kappa=0.75 t0=10.0 init_buffer=75 term_buffer=50 window=25 algorithm=hmc engine=nuts max_depth=10 metric=diag_e stepsize=1.0 stepsize_jitter=1.0 random seed=-1 id=1 data file=bernoulli_1.data.R output file=bernoulli_samples_1.csv refresh=100`
 `./bernoulli sample num_samples=1000 num_warmup=1000 save_warmup=0 thin=1 adapt engaged=1 gamma=0.05 delta=0.85 kappa=0.75 t0=10.0 init_buffer=75 term_buffer=50 window=25 algorithm=hmc engine=nuts max_depth=10 metric=diag_e stepsize=1.0 stepsize_jitter=1.0 random seed=-1 id=2 data file=bernoulli_2.data.R output file=bernoulli_samples_2.csv refresh=100`
 `./bernoulli sample num_samples=1000 num_warmup=1000 save_warmup=0 thin=1 adapt engaged=1 gamma=0.05 delta=0.85 kappa=0.75 t0=10.0 init_buffer=75 term_buffer=50 window=25 algorithm=hmc engine=nuts max_depth=10 metric=diag_e stepsize=1.0 stepsize_jitt

Check for success and examine results. The `simulated` object is of type `Mamba.Chains`.

In [ ]:
if rc == 0
  sim = simulated[1:1000, ["lp__", "theta", "accept_stat__"], :]
  describe(sim)
end

In [12]:
simulated.stan_summary()

LoadError: [91mtype Chains has no field stan_summary[39m

In [ ]:
println("Brooks, Gelman and Rubin Convergence Diagnostic")
try
  gelmandiag(sim, mpsrf=true, transform=true) |> display
catch e
  #println(e)
  gelmandiag(sim, mpsrf=false, transform=true) |> display
end

In [ ]:
println("Geweke Convergence Diagnostic")
gewekediag(sim) |> display

In [ ]:
println("Highest Posterior Density Intervals")
hpd(sim) |> display

In [ ]:
println("Cross-Correlations")
cor(sim) |> display

In [ ]:
println("Lag-Autocorrelations")
autocor(sim) |> display

In [ ]:
p = plot(sim, [:trace, :mean, :density, :autocor], legend=true);
draw(p, ncol=4, filename="summaryplot", fmt=:svg)
draw(p, ncol=4, filename="summaryplot", fmt=:pdf)